# Simulation with AR data

In [1]:
%load_ext lab_black

In [2]:
cd ../../

ERROR:root:Cannot parse: 1:4: cd ../../
Traceback (most recent call last):
  File "/home/adam2392/Documents/morf-demo/.venv/lib/python3.8/site-packages/lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "/home/adam2392/Documents/morf-demo/.venv/lib/python3.8/site-packages/lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "/home/adam2392/Documents/morf-demo/.venv/lib/python3.8/site-packages/black/__init__.py", line 974, in format_str
    src_node = lib2to3_parse(src_contents.lstrip(), mode.target_versions)
  File "/home/adam2392/Documents/morf-demo/.venv/lib/python3.8/site-packages/black/__init__.py", line 1083, in lib2to3_parse
    raise exc from None
black.InvalidInput: Cannot parse: 1:4: cd ../../


/home/adam2392/Documents


In [3]:
import pandas as pd
from scipy import stats
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

%matplotlib inline
import seaborn as sns

import pickle
import time
from tqdm import tqdm

from sklearn.model_selection import train_test_split

# comparative classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

# evaluation classifier
from rerf.rerfClassifier import rerfClassifier

import sys

print(sys.version)
import warnings

warnings.simplefilter("ignore")

%matplotlib inline
%load_ext autoreload
%autoreload 2

3.8.3 (default, May 19 2020, 18:47:26) 
[GCC 7.3.0]


In [4]:
seed = 12345

In [5]:
# length of data sequence
ns = np.array([50, 100, 200, 400, 1000])

# assume you can train on half trials and
# test on half the trials
test_size = 0.5

# number of samples
n_samples = 100

# simulated data parameters
n_trials = 100

# dimensionality of data and noise
data_dim = 3
noise_dim = 3
n_chs = data_dim + noise_dim

snr_factor = 1.0

# data mean vector
mu = np.zeros((data_dim,))

# permutation strategy
permutation_strategy = "alternate"

# Simulation of AR(1) Data

In [ ]:
def _generate_random_input_pulse(n_chs, n_steps):
    # generate a C x T array
    ut = np.zeros((n_chs, n_steps))
    
    # randomly select a time point around the center
    # center_time = n_steps // 2
    K = 20
    rand_ind = np.random.randint(-K, K, size=None)
    ut[:, rand_ind:rand_ind+2] = 1
    
    # ut[:, center_time:center_time+2] = 1
    
    return ut

def generate_ar_samples2(cls=0, n_steps=1000, k=1):
    """
    Generate multivariate time-series for 3 channels of length n_samples
    from a vector AR(1) model. Uncorrelated white noise channels are
    interspersed between the vector sequences.

    VAR(1) Model: 
    y(t+1) = Ay(t) + Bu(t) + err(t)
    """
    n_chs = 3
    n_noise_chs = 3
    
    # scale data along time w/ normalization
    scaler = StandardScaler()

    A = np.array([
        [0.5,  0.5*k, 0.7*k],
        [0.3*k, -0.8, -0.15*k],
        [0.5*k,  0.2*k, -0.6],
    ])
        
    # signal that separates binary classes
    if cls == 0:
        B = np.array([
            [  0.5,  0.1*k, 0.1*k],
            [0, 0.5, 0.1*k],
            [0,  0, -0.6]
        ])
    elif cls == 1:
        B = np.array([
            [  0.5,  0, 0],
            [0, 0.5, 0],
            [0,  0, -0.6]
        ])
    
    # create DT Linear system
    linsys = DiscreteLinearSystem(A=A, B=B)
    
    # generate input
    ut = _generate_random_input_pulse(n_chs, n_steps)
    
    # generate random initial condition and simulate what system would look like
    x0 = np.random.random((n_chs,))
    y = linsys.reconstruct(x0, ut, steps=n_steps, add_noise=False)
    assert y.shape == (n_chs, n_steps)
    
    # add iid noise to the system simulation
    y_noise = np.random.normal(scale=0.5, size=(n_noise_chs, n_steps))  # add Gaussian noise?
    # print(f"max: {np.max(y)}, min: {np.min(y)}")
    y = y + y_noise
    
    # stack noise 
    y_noise = np.random.random((n_noise_chs, n_steps))
    y = np.vstack((
        y[0,:], y_noise[0,:],
        y[1,:], y_noise[1,:],
        y[2,:], y_noise[2,:]
    ))
    
    # normalize along the time domain
    # y = scaler.fit_transform(y.T).T
    return y

In [4]:
def generate_ar_samples(cls=0, n_steps=1000, k=1):
    """
    Generate multivariate time-series for 3 channsl of length n_samples
    from a vector AR(1) model. Noise channels are interleaved between
    the vector sequences.

    VAR(1) Model: 
    y(t+1) = Ay(t) + Bu(t) + err(t)

    For now, we ignore exogenous variables, i.e. B = 0, and assume
    uncorrelated errors.
    """

    n_chs = 3
    n_noise_chs = 3

    A = np.array([[
    [1.0,    0.5,    0.5],
    [  0.4, 0.9,    0.5],
    [  0.4,    0.4, 0.8]
    ]])
    if cls == 0:
        B = []
#     if cls == 0:
#         A = np.array([[
#             [  0.5,  0.5*k, 0.7*k],
#             [0.3*k, -0.8, -0.15*k],
#             [0.5*k,  0.2*k, -0.6]
#         ]])
#     elif cls == 1:
#         A = np.array([[
#             [1.0,    0.5,    0.5],
#             [  0.4, 0.9,    0.5],
#             [  0.4,    0.4, 0.8]
#         ]])

    B = np.zeros((n_chs, n_chs))

    A_eigvals = np.linalg.eigvals(A)
    A_spec_rad = np.max(np.abs(A_eigvals))

    # TODO: Figure out how to best set error relative to A matrix
    err_cov = 0.05 * A_spec_rad * np.identity(n_chs)

    var = VARProcess(A, B, err_cov, _params_info={'k_trend': 0})
    y = var.simulate_var(steps=n_steps).T
    assert y.shape == (n_chs, n_steps)

    y_noise = np.random.random((n_noise_chs, n_steps))

    y = np.vstack((
        y[0,:], y_noise[0,:],
        y[1,:], y_noise[1,:],
        y[2,:], y_noise[2,:]
    ))
    
    return y

In [ ]:
def simulate_data(ns, T, nchs=6, cov_factor=1, test_size=0.5, random_state=None):
    # initialize data structures for train/test data
    X_train, Y_train = np.empty(shape=(0,nchs,T)), np.empty(shape=(0))
    X_test, Y_test = np.empty(shape=(0,nchs,T)), np.empty(shape=(0))

    # simulate over varying sizes of data sequence
    for n in ns:
        y = []
        X = []

        # generate correlated multi-variate time series
        for i in range(n):
            _x = generate_ar_samples2(cls=0, n_steps=T, k=cov_factor)
            X.append(_x)
            y.append(0)
        for i in range(n):
            _x = generate_ar_samples2(cls=1, n_steps=T, k=cov_factor)
            X.append(_x)
            y.append(1)
        X = np.array(X)
        y = np.array(y)

        # perform training test split
        _X_train, _X_test, _y_train, _y_test = train_test_split(X, y, 
                                                            test_size=test_size, 
                                                            random_state=random_state)
        X_train = np.vstack((X_train, _X_train))
        Y_train = np.hstack((Y_train, _y_train))
        X_test = np.vstack((X_test, _X_test))
        Y_test = np.hstack((Y_test, _y_test))

    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)
    return X_train, Y_train, X_test, Y_test

In [ ]:
T = 100  # number of time steps for each sequence
ns = np.array([50,100,200,400,1000,2000])  # length of data sequence
nchs = 6  # number of channels in each MTS
test_size = 0.5  # size of the test data
random_state = 1234  # random state to separate training/testing samples
np.random.seed(random_state)

X_train, Y_train, X_test, Y_test = simulate_data(
    ns, T, nchs=nchs, cov_factor=1, test_size=test_size, random_state=random_state
)
print(X_train.shape, Y_train.shape, X_test.shape)